# Country project (case data)

## Reads .cvs file for region names, region populations, and Matrix 1

Run the cells below first.

In [36]:
import pandas as pd
import os; os.chdir("/Users/gloftus/covid-king/")

rows_cols = (2, 5)
filename = "figs/country_cases.png"
Geofffile = "data/RelevantStates.cases.csv"


In [37]:
from pathlib import Path
import datetime
import io
import ast
import warnings

import requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
import pandas as pd; pd.set_option('display.max_rows', 500); pd.set_option('display.float_format', '{:.10f}'.format)
from sklearn.linear_model import LinearRegression

# Get Regions, populations, Matrix1 data from relevant .csv file
# print(TestMatrix1.head())

## GENERATE THE PLOT

Instructions:
---
- pass in a list of ISOs at the top
- if desired, pass in the number of rows and columns you want in the figure with `rows_cols = (rownumber, columnnumber)`
- if `rows_cols = (None, None)`, then the script will default to 4 columns and a dynamic number of rows.

In [55]:
def get_matrix2(matrix1, populations, data):
    print(populations/1000000, sep='\t')
    return matrix1.diff().rolling(14).mean() / (populations/1000000)

def get_matrix3(matrix2):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return np.log10(matrix2)

def linear_regression(country_col, future_day_count):
    # pred_idx is a list of the next `future_day_count` days, starting tomorrow.
    # Each of the elements are dates, not numbers.
    pred_idx = pd.date_range(start=country_col.index[-1] + pd.Timedelta(days=1), 
                             periods=future_day_count, freq="D")
    
    # days_from_start is a list of numbers, each of which denotes one of the previous 7 days,
    # starting with 0 (0 representing 7 days ago)
    days_from_start = (country_col.index - country_col.index[0]).days
    
    # the integer-day index for tomorrow.
    tomorrow = days_from_start[-1] + 1
    
    # the integer-day indices for the days we want to predict.
    to_pred = np.arange(tomorrow, tomorrow + future_day_count).reshape(-1, 1)
    
    # Get the pure integer indices for the days that we're fitting the linear regression on,
    # in integers, and then format properly
    X = days_from_start.values.reshape(-1, 1)
    
    # get the actual values for the days we're fitting the linear regression on, then format properly
    Y = country_col.values.reshape(-1, 1)
    
    # Fit out linear regression model on the previous 7 days, then predict the next `future_day_count` days
    y_pred = LinearRegression().fit(X, Y).predict(to_pred)
    
    # Give back the dates that we predicted, and the predicted values
    return pred_idx, y_pred.squeeze()

def predict_future(df, future_day_count=14):
    last_7_days = df[-7:]
    df_dict = {}
    for col in last_7_days:
        series = last_7_days[col]
        idx, pred = linear_regression(series, future_day_count=future_day_count)
        df_dict[col] = pred

    return 10**pd.DataFrame(df_dict, index=idx)

def main(rows_cols=(None, None), filename=""):
    
    # grab matrices
    """
    Extract data from rows 2-n of Test.Matrix.csv 
    Anything below involving iso codes is irrelevant
    """   
    df = pd.read_csv('Test.Matrix.csv', index_col=0)
    populations = df.iloc[0:1]
    
    matrix1 = df.iloc[2:len(df)]
    matrix2 = get_matrix2(matrix1, populations, data=df)
    matrix3 = get_matrix3(matrix2)
    final = 10**matrix3
    
    # add regression prediction
    regression_predictions = predict_future(matrix3, future_day_count=14)
    
    # --- Plotting --- #
    
    # By default, create the grid dynamically with 4 columns and changing 
    # number of rows. If you want, you can pass a specific number of rows
    # and columns by passing (rows_cols) to this function.
    n_plots = len(isos)
    if rows_cols == (None, None):
        ncols = 4
        quotient, remainder = divmod(n_plots, ncols)
        nrows = quotient if remainder==0 else quotient+1
    else:
        nrows, ncols = rows_cols
        quotient, remainder = divmod(n_plots, ncols)
        
    # set up figure by making an empty grid of plots, each of which will get 
    # data added to it in the country loop
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*3.5))
        
    # get rid of any plots we aren't using
    if remainder != 0:
        diff = len(axs.flat) - len(isos)
        for ax in axs.flat[-diff:]:
            ax.set_axis_off()
    
    # loop through the countries we're plotting, and make a 
    # plot for each country
    for i, iso in enumerate(isos):
        
        # get the name of the country and create a scatterplot
        country_name = iso_mapping[iso]
        ax = axs.flat[i]
        g = sns.scatterplot(data=final, x=final.index, y=country_name, 
                            ax=ax, color="black", edgecolor="none", s=8, )
        
        # add regression line to the current country plot and make blue
        sns.lineplot(data=regression_predictions, x=regression_predictions.index, 
                     y=country_name, ax=g, color="blue")
        
        
        # add title and set y-axis to logarithmic scale
        g.set_title(f"{country_name}", fontsize=20)
        g.set(yscale="log", ylabel="NCPC", ylim=(1, 2000))
        
        # mess with x-axis ticks
        g.xaxis.set_minor_locator(mdates.WeekdayLocator(interval=2))  # change the interval for the smaller ticks on x-axis to every 2 days
        g.xaxis.set_major_locator(mdates.WeekdayLocator(interval=8))  # change the interval for the larger ticks on x-axis to every 8 days
        g.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))  # show x-ticks in month-day format, don't display years
        g.xaxis.label.set_size(14)  # sets size of x-axis label
        g.tick_params(axis='x', labelrotation=45)  # rotate ticks by 45 degrees
        
        # mess with y-axis ticks
        g.yaxis.set_major_formatter(ScalarFormatter())  # ticks in integers rather than scientific notation
        g.yaxis.label.set_size(14)  # sets size of y-axis label
        
        # add green horizontal line
        g.axhline(y=7, color="green")
        
        # add horizontal grid lines, make the color black, give them transparency of 0.1
        g.yaxis.grid(which='major', color='black', alpha=0.2)
        
    # Increase the vertical (height) distance between plots on the grid
    plt.tight_layout(h_pad=2.5)
    
    # save the figure
    if filename:
        plt.savefig(filename, bbox_inches="tight", dpi=144)
    
# Actually run the program
main(isos, rows_cols=rows_cols, filename=filename)  

TypeError: main() got multiple values for argument 'rows_cols'

# Dad Playground

In [60]:
df = pd.read_csv('Test.Matrix.csv', index_col=0)
populations = df.iloc[0:1]
populations
# matrix1 = df.iloc[2:len(df)]

,Massachusetts,New York,Maryland,North Carolina,Florida,Penn,Indiana,Illinois,Nevada,Washington,Oregon,Nothern California,Southern California,Canada,Italy,United Kingdom
date,6.94,19.45,6.05,10.49,21.48,12.8,6.73,12.67,3.08,7.61,4.22,15.73,23.64,38.01,60.32,67.89


## Utility: finding ISO codes

Use the two cells below to find ISOs from country names or vice-versa.
- the `iso_mapping` variable contains the entire list of mappings

In [38]:
# You can get isos from countries with
# isos = list(country_to_iso(["Ireland", "United Kingdom", "Portugal", "Spain", "France", 'Italy', 'Belgium', 'Luxembourg']))
# isos = list(country_to_iso(["Ireland", "United Kingdom", "Portugal", "Spain", "France", "Italy", "Belgium", "Luxembourg", "Denmark"]))

# Or just pass in a single country:
# isos = isos_from_countries = list(country_to_iso(["Brazil", "Argentina","Chile", "Peru", "Bolivia", "Paraguay", "Uruguay", "Venezuela", "Colombia"]))
isos = isos_from_countries = list(country_to_iso(["Venezuela", "Colombia", "Bolivia", "Ecuador", "Peru", "Brazil", "Argentina", "Paraguay", "Uruguay", "Chile"]))
#  isos = isos_from_countries = list(country_to_iso(["Venezuela", "Colombia", "Bolivia", "Peru", "Brazil", "Argentina", "Paraguay", "Uruguay", "Chile"]))

print(isos)

['VEN', 'COL', 'BOL', 'ECU', 'PER', 'BRA', 'ARG', 'PRY', 'URY', 'CHL']


In [7]:
# You can get countries from isos with
country_names = list(iso_to_country(isos))
print(country_names)

# Or a single iso:
iso_to_country("GBR")

['Ireland', 'United Kingdom', 'Portugal', 'Spain', 'France', 'Italy', 'Belgium', 'Luxembourg', 'Denmark']


'United Kingdom'

# Notes from when I made the functions all separately

## Get data for daily regions (cases)

The below code will grab the current data and cache it into a local .csv file.  

If you already have the local csv file and it's up-to-date (meaning, the most recent date in it is greater than or equal to yesterday), just use the local csv file to avoid re-downloading uneccesarily.

If you don't already have the local csv file, download it as well.

**useful things**
- check out the variable `iso_mapping`. It maps iso codes to country names.
- You can use this to look at particular country codes. For example, you could type `iso_mapping[['HUN', 'EST']]` to look at the iso conversions for Hungary and Estonia.

## Matrix 1

The below code creates Matrix 1 for each cell. Feel free to peruse it. You just pass a list of ISO codes into `convert`. More information in the string underneath the `convert` definition (`def`).

**useful things**
- the `iso_to_country` function eats an ISO and outputs a country name. For example, iso_to_country("IRL") returns "Ireland".
- the `country_to_iso` function eats a country and outputs an ISO. For example, country_to_iso("Ireland") returns "IRL".

## Matrix 2

This code calculates matrix 2. There are small disparity between these numbers and the ones in the Excel spreadsheet -- that's because I calculated population manually based on the original countries_data, rather than using the numbers in the sheet. 

If you'd like you can replace `populations = np.array([get_population(country) for country in iso_list])` with `populations = np.array([4.977, 67.886, 10.296, 47.431, 67.081, 60.317, 11.493, 0.626])` for e.g. EU1. 

I personally think this is too much of a hassle.

## Matrix 4

The fun part!

Thoughts:
- Lasso Regression or Elastic Regression is almost always better to use than normal linear regression. I can swap those out very easily if you'd like.

In [1]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

us_state_abbrev_r = {v: k for k, v in us_state_abbrev.items()}